# TAREA DE EJERCICIO

Actualiza el proyecto del día 1 para resumir una página web y utilizar un modelo de código abierto que se ejecute localmente a través de Ollama en lugar de OpenAI

Podrás utilizar esta técnica para todos los proyectos posteriores si prefiere no utilizar API de pago (closed source).

**Beneficios:**
1. Sin cargos por API: código abierto
2. Los datos no salen de su ordenador

**Desventajas:**
1. Tiene mucha menos potencia (parámetros) que el modelo Frontier

## Resumen de la instalación de Ollama

¡Simplemente visita [ollama.com](https://ollama.com) e instálalo!

Una vez que hayas terminado, el servidor ollama ya debería estar ejecutándose localmente.
Si entras en:
[http://localhost:11434/](http://localhost:11434/)

Debería ver el mensaje `Ollama se está ejecutando`.

De lo contrario, abre una nueva Terminal (Mac) o Powershell (Windows) e introduce `ollama serve`.
Luego, intenta entrar em [http://localhost:11434/](http://localhost:11434/) nuevamente.

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constantes

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Crea una lista de mensajes utilizando el mismo formato que usamos para OpenAI

messages = [
    {"role": "user", "content": "Describe algunas de las aplicaciones comerciales de la IA generativa."}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [5]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

La inteligencia artificial generativa (IA) tiene una amplia gama de aplicaciones comerciales que pueden revolucionar la forma en que los negocios operan y interactúan con sus clientes. A continuación, se presentan algunas de las aplicaciones comerciales más destacadas de la IA generativa:

1. **Personalización de productos y experiencias**: La IA generativa puede ser utilizada para crear productos personalizados según las preferencias y necesidades de los clientes. Por ejemplo, una tienda de ropa puede utilizar la IA para diseñar vestimenta ajustada a medida según el cliente.
2. **Creación de contenido**: La IA generativa puede generar contenido como texto, imágenes y videos que se pueden utilizar en campañas publicitarias, redes sociales y sitios web. Esto puede ayudar a ahorrar tiempo y dinero en la creación de contenido humano.
3. **Automatización de tareas administrativas**: La IA generativa puede ser utilizada para automatizar tareas administrativas como la generación de informes,

# Presentación del paquete ollama

Ahora haremos lo mismo, pero utilizando el elegante paquete de Python ollama en lugar de una llamada HTTP directa.

En esencia, se realiza la misma llamada que se indicó anteriormente al servidor ollama que se ejecuta en localhost:11434

In [ ]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

# AHORA el ejercicio para ti

Toma el código del día 1 e incorpóralo aquí para crear un resumidor de sitios web que use Llama 3.2 ejecutándose localmente en lugar de OpenAI

In [6]:
# imports
import os
import requests
from IPython.display import Markdown, display
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import ollama

In [7]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No se encontró ninguna clave API: diríjase al cuaderno de resolución de problemas en esta carpeta para identificarla y solucionarla.")
elif not api_key.startswith("sk-proj-"):
    print("Se encontró una clave API, pero no inicia sk-proj-; verifique que esté usando la clave correcta; consulte el cuaderno de resolución de problemas")
elif api_key.strip() != api_key:
    print("Se encontró una clave API, pero parece que puede tener espacios o caracteres de tabulación al principio o al final; elimínelos; consulte el cuaderno de resolución de problemas")
else:
    print("¡Se encontró la clave API y hasta ahora parece buena!")


¡Se encontró la clave API y hasta ahora parece buena!


In [8]:
class Website:
    """
    Una clase de utilidad para representar un sitio web que hemos scrappeado
    """

    def __init__(self, url):
        """
        Crea este objeto de sitio web a partir de la URL indicada utilizando la biblioteca BeautifulSoup
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No tiene título"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [9]:
frog = Website("https://cursos.frogamesformacion.com")
print(frog.title)
print(frog.text)

Frogames
Ir al contenido principal
Frogames
Menú alternativo
Menú
Iniciar sesión
Ganadora del premio 'Enseñanza online de datos y competencias digitales más innovadora de Europa, 2023'
Pasión por
aprender
con los
mejores
En Frogames Formación te ayudamos a convertirte en todo un experto en: Programación de Videojuegos, Inteligencia Artificial, Machine Learning, Desarrollo de Apps, Data Science y mucho más.
Aprende mientras te diviertes
Cursos, Rutas y Suscripciones
Certificados de finalización
Qué encontrarás
dentro
de Frogames
Cursos online y formación de calidad para toda la família
Rutas temáticas
Rutas organizadas para que aprendas paso a paso, subiendo cada escalón e incrementando tus conocimientos adquiridos
Instructores Expertos
Con un equipo de profesionales y expertos en la materia que te acompañará a lo largo de todo el aprendizaje en la plataforma
Certificados blockchain
Títulos verificados por blockchain para cada habilidad que aprendas, listos para compartir en tus redes s

In [10]:
system_prompt = "Eres un asistente que analiza el contenido de un sitio web \
y proporciona un breve resumen, ignorando el texto que podría estar relacionado con la navegación. \
Responder en Markdown."

In [11]:
def user_prompt_for(website):
    user_prompt = f"Estás viendo un sitio web titulado {website.title}"
    user_prompt += "\nEl contenido de este sitio web es el siguiente; \
    proporciona un breve resumen de este sitio web en formato Markdown. \
    Si incluye noticias, productos o anuncios, resúmelos también.\n\n"
    user_prompt += website.text
    return user_prompt

In [12]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [14]:
def summarize(url):
    website = Website(url)
    response = ollama.chat(model=MODEL, messages=messages_for(website))
    return response['message']['content']

In [15]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [16]:
display_summary("https://cursos.frogamesformacion.com")

¡Hola! Me parece que has proporcionado un texto largo y detallado sobre la plataforma de formación en línea "Frogames". Aquí te presento una síntesis de los puntos más importantes:

**Sobre Frogames**

* Froggames es una plataforma de formación en línea que ofrece cursos en diferentes temas, como matemáticas, lenguajes de programación, emprendimiento, análisis de datos, inteligencia artificial y desarrollo de videojuegos.
* La plataforma fue fundada por Juan Gabriel Gomila, un licenciado en matemáticas y maestro en educación.

**Características de la plataforma**

* Más de 500.000 estudiantes han sido formados en Frogames en habla hispana.
* El equipo docente consta de más de 20 instructores online.
* La plataforma ofrece cursos gratuitos y pagos para empresas.
* Puedes optar a convertirte afiliado y ser remunerado por cada venta que consigas.

**Temas de aprendizaje**

* C#,
* Excel,
* Habilidades Blandas,
* Java,
* Kotlin,
* Machine Learning,
* Marketing,
* Power BI,
* Python,
* R,
* Reporting,
* SQL,
* Solidity,
* Swift,
* Tableau,
* Unity,
* Unreal Engine,
* Web.

**Premios y distinciones**

* Frogames Formación y Juan Gabriel Gomila han ganado varios premios y distinciones, como plataforma educativa destacada a nivel mundial.

**Política de privacidad y uso**

* Puedes encontrar información sobre la política de privacidad, términos de uso, política de cookies y política de redes sociales en las páginas correspondientes de la plataforma.

Espero que esta síntesis te haya sido útil. ¿Hay algo específico que quieras saber o discutir sobre Frogames?